In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("btc_price_prediction") \
    .config(
        "spark.jars.packages",
        "org.postgresql:postgresql:42.7.3"
    ) \
    .getOrCreate()




In [19]:
#spark.stop()

## 1. Data Injection

In [23]:
import requests
import os
from dotenv import load_dotenv
load_dotenv() 
def data_collection():

  SYMBOL = 'BTCUSDT'
  INTERVAL = '1m'   # Intervalle d'une minute
  LIMIT = 600    
  api='https://api.binance.com/api/v3/klines'
  response = requests.get(api, params={
    "symbol": SYMBOL,
    "interval": INTERVAL,
    "limit": LIMIT
  })
  if response.status_code != 200:
        print(f"Erreur {response.status_code}")
        return None
  api_data=response.json()
  columns = [
        "open_time", "open_price", "high_price", "low_price", "close_price", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ]
  psdf= spark.createDataFrame(api_data, columns)
 
  
  return psdf
  
data = data_collection()
data.show()

+-------------+--------------+--------------+--------------+--------------+-----------+-------------+------------------+----------------+---------------------+----------------------+------+
|    open_time|    open_price|    high_price|     low_price|   close_price|     volume|   close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------------+--------------+--------------+--------------+--------------+-----------+-------------+------------------+----------------+---------------------+----------------------+------+
|1768878600000|92426.69000000|92426.69000000|92397.03000000|92410.24000000| 5.15707000|1768878659999|   476560.14659830|            1889|           1.24753000|       115271.75277680|     0|
|1768878660000|92410.23000000|92439.84000000|92410.23000000|92439.84000000| 2.21993000|1768878719999|   205180.43060550|             929|           1.97002000|       182081.88756220|     0|
|1768878720000|92439.84000000|92439.84000000|92402

In [24]:
data.printSchema()

root
 |-- open_time: long (nullable = true)
 |-- open_price: string (nullable = true)
 |-- high_price: string (nullable = true)
 |-- low_price: string (nullable = true)
 |-- close_price: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: string (nullable = true)
 |-- number_of_trades: long (nullable = true)
 |-- taker_buy_base_volume: string (nullable = true)
 |-- taker_buy_quote_volume: string (nullable = true)
 |-- ignore: string (nullable = true)



In [ ]:
""""
from pyspark.sql.functions import col
import pyspark.sql.functions as sf
num_cols= ["open_price", "high_price", "low_price", "close_price", "volume", "quote_asset_volume", "number_of_trades", "taker_buy_base_volume", "taker_buy_quote_volume"]
data = data.select([col(feature).cast('double') for feature in num_cols])
data= data.select([col("open_time"), sf.timestamp_millis("open_time")])
data.printSchema()
"""




In [31]:
from pyspark.sql.functions import col
num_cols= ["open_price", "high_price", "low_price", "close_price", "volume", "quote_asset_volume", "number_of_trades", "taker_buy_base_volume", "taker_buy_quote_volume"]
for feature in num_cols:
      data = data.withColumn(feature, col(feature).cast('double'))

data.printSchema()

root
 |-- open_time: long (nullable = true)
 |-- open_price: double (nullable = true)
 |-- high_price: double (nullable = true)
 |-- low_price: double (nullable = true)
 |-- close_price: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: double (nullable = true)
 |-- number_of_trades: double (nullable = true)
 |-- taker_buy_base_volume: double (nullable = true)
 |-- taker_buy_quote_volume: double (nullable = true)
 |-- ignore: string (nullable = true)



In [34]:
from pyspark.sql.functions import from_unixtime , to_timestamp

data = data.withColumn("open_time_ts", to_timestamp(from_unixtime(col("open_time")/1000))) \
       .withColumn("close_time_ts", to_timestamp(from_unixtime(col("close_time")/1000)))
data.printSchema()
data.show(5)


root
 |-- open_time: long (nullable = true)
 |-- open_price: double (nullable = true)
 |-- high_price: double (nullable = true)
 |-- low_price: double (nullable = true)
 |-- close_price: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- close_time: long (nullable = true)
 |-- quote_asset_volume: double (nullable = true)
 |-- number_of_trades: double (nullable = true)
 |-- taker_buy_base_volume: double (nullable = true)
 |-- taker_buy_quote_volume: double (nullable = true)
 |-- ignore: string (nullable = true)
 |-- open_time_ts: timestamp (nullable = true)
 |-- close_time_ts: timestamp (nullable = true)

+-------------+----------+----------+---------+-----------+-------+-------------+------------------+----------------+---------------------+----------------------+------+-------------------+-------------------+
|    open_time|open_price|high_price|low_price|close_price| volume|   close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume